In [17]:
#10(1)

import numpy as np

# if bins=[v1,v2,v3,v4],
# v1 < 1st interval(bin) <= v2
# v2 < 2nd interval(bin) <= v3
# v3 < 3rd interval(bin) <= v4
# Equal width discretization function


def equal_width(data, num_bins):
    min = np.min(data)
    max = np.max(data)
    # hint: may use numpy's linspace
    bins = np.linspace(min, max, num_bins + 1)
    return bins


# Continuous data
data = np.array([9.2, 6.1, 5.7, 4.1, 2.2, 7.3, 8.6, 9.9, 8.6])

# Define the number of bins
num_bins = 3 # Suppose num_bins=3

# compute intervals using equal width function
bins = equal_width(data, num_bins)

# Using bins, discretize the data by assigning it to bins
disc_data = np.digitize(data, bins, right=True)

# Display the results
# show the results of original data, bins, disc_data, etc
print("original data:", data)
print("bins:", bins)
print("disc_data:", disc_data)


original data: [9.2 6.1 5.7 4.1 2.2 7.3 8.6 9.9 8.6]
bins: [2.2        4.76666667 7.33333333 9.9       ]
disc_data: [3 2 2 1 0 2 3 3 3]


In [18]:
#10(2)

import numpy as np

# (input) y: array of target values
# (output) entropy: entropy value of y
# for simplicity. assume binary class only
# e.g., [0,0,1,0,0,1,1,1]

def calculate_entropy(y):
    class_counts = np.bincount(y)
    probabilities = class_counts/len(y)
    probabilities = probabilities[probabilities > 0]
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

# (input) y: array of target values
# y_left: left interval
# y_right: right interval
# e.g., y=[1,2,3,4,5], y_left=[1,2], y_right=[3,4,5]
# (output) info_gain: information gain between entropy(y) and average entropy after split.

def information_gain(y, y_left, y_right):
    entropy_before = calculate_entropy(y)
    entropy_left = calculate_entropy(y_left)
    entropy_right = calculate_entropy(y_right)
    weighted_entropy_after = ((len(y_left) / len(y)) * entropy_left + (len(y_right) / len(y)) * entropy_right)
    info_gain = entropy_before - weighted_entropy_after
    return info_gain

# compute entropy of y
# calculate entropy of y_left and y_right, respectively
# compute information gain

def best_split(X, y):
    best_info_gain = -1
    best_split_point = None
    
    for value in np.unique(X):
        y_left = y[X <= value]
        y_right = y[X > value]
        gain = information_gain(y, y_left, y_right)

        if gain > best_info_gain:
            best_info_gain = gain
            best_split_point = value
            
    return best_split_point

#initialize best information gain value=-1 & best split point=None
# y_left : interval with y values <=value
# y_right: interval with y values > value
# compute information gain using two intervals y_left & y_right
# update best_info & best_split_point

X = np.array([1.5, 4.5, 2.0, 3.5, 1.5, 7.5, 3.5, 5.0])
y = np.array([0, 0, 1, 0, 0, 1, 1, 1]) #binary

# Best split point
split_point = best_split(X, y)
print("The best split point in a numeric data array:", split_point)


The best split point in a numeric data array: 1.5


In [19]:
#11. Apriori algorithm

import itertools

trans_db=[ ['milk','bread','biscuit'], ['bread','milk','biscuit','cornflakes'],
['bread','tea','bournvita'], ['jam','maggi','bread','milk'], ['maggi','tea','biscuit'],
['bread','tea','bournvita'], ['maggi','tea','cornflakes'], ['maggi','bread','tea','biscuit'],
['jam','maggi','bread','tea'], ['bread','milk'], ['coffee','cock','biscuit','cornflakes'],
['coffee','cock','biscuit','cornflakes'], ['coffee','sugar','bournvita'], ['bread','coffee','cock'],
['bread','sugar','biscuit'], ['coffee','sugar','cornflakes'], ['bread','sugar','bournvita'],
['bread','coffee','sugar'], ['bread','coffee','sugar'], ['tea','milk','coffee','cornflakes'] ]

min_support = 3

# infreq_list : the set of infrequent itemsets (itemsets whose support is less than min_support)
# 2D list structure. e.g., [ [‘bread’], [‘sugar’], [‘coffee’, bisciut’], ....]
infreq_itemsets=[]
# compute support value of an itemset
def compute_support(itemset):
    support=0
    for trans in trans_db:
        if set(itemset).intersection(set(trans)) == set(itemset):
            support = support + 1
    return support

# Generate k+1 frequent itemsets from k frequent itemsets
# (input) k_itemsets: k itemsets (itemset with length=k)
# 2D list (length k). e.g. (k=2), [ [‘bread’, ‘sugar’], [‘coffee’, ‘bisciut’], ....]
# (output) k_1_itemsets: (k+1) itemsets with length=k+1
# 2D list (length k+1). e.g. (length=3), [ [‘bread’, ‘sugar’, ‘coffee’], [‘coffee’, ‘bisciut’, ‘jam’], ....]
def generate_k_1_itemsets(k_itemsets):
    k_1_itemsets = []
    
    k_itemsets = [itemset for itemset in k_itemsets if compute_support(itemset) >= min_support]
    
    for i in range(len(k_itemsets)):
        for j in range(i + 1, len(k_itemsets)):
            new_itemset = sorted(list(set(k_itemsets[i]).union(set(k_itemsets[j]))))
            
            if len(new_itemset) == len(k_itemsets[0]) + 1:
                if compute_support(new_itemset) >= min_support:
                    if new_itemset not in k_1_itemsets:
                        k_1_itemsets.append(new_itemset)
                else:
                    if new_itemset not in infreq_itemsets:
                        infreq_itemsets.append(new_itemset)
    
    return k_1_itemsets

# In k_itemsets, remove itemsets whose support < min_support
# k_i_itemsets: frequent (k_+1) itemsets (support >= min_support)
# create ‘new_itemset’ by combining i and J
# skip ‘new_itemset’ if
# 1) i == j or
# 2) ‘new_itemset’ is in in ‘infreq_itemsets’ or
# 3) length is not (k+1) or
# 4) support of ‘new_itemset’ < min_support
# original 1-items (wihout considering min_support values)


all_list_items = list(set(item for sublist in trans_db for item in sublist))

# Frequent 1-itemsets, 2-itemsets, 3-itemsets
item_1 = [[item] for item in all_list_items if compute_support([item]) >= min_support]
print("Frequent 1-itemsets:\n", item_1)

item_2 = generate_k_1_itemsets(item_1)
print("\nFrequent 2-itemsets:\n", item_2)

item_3 = generate_k_1_itemsets(item_2)
print("\nFrequent 3-itemsets:\n", item_3)

Frequent 1-itemsets:
 [['bread'], ['cock'], ['bournvita'], ['milk'], ['biscuit'], ['tea'], ['maggi'], ['coffee'], ['cornflakes'], ['sugar']]

Frequent 2-itemsets:
 [['bournvita', 'bread'], ['bread', 'milk'], ['biscuit', 'bread'], ['bread', 'tea'], ['bread', 'maggi'], ['bread', 'coffee'], ['bread', 'sugar'], ['cock', 'coffee'], ['biscuit', 'cornflakes'], ['maggi', 'tea'], ['coffee', 'cornflakes'], ['coffee', 'sugar']]

Frequent 3-itemsets:
 []


In [20]:
#12

from sklearn.datasets import load_iris
from sklearn.preprocessing import KBinsDiscretizer

data = load_iris()
features = data.data
target_names = data.feature_names

for method, method_label in zip(['uniform', 'quantile'], ['Equal Width', 'Equal Frequency']):
    print(f'\n{method_label} Discretization:')
    
    discretizer = KBinsDiscretizer(n_bins = 5, encode = 'ordinal', strategy = method, subsample = None)
    discretizer.fit(features)
    for feature_name, bins in zip(target_names, discretizer.bin_edges_):
        print(f"Feature {feature_name}: {bins}")



Equal Width Discretization:
Feature sepal length (cm): [4.3  5.02 5.74 6.46 7.18 7.9 ]
Feature sepal width (cm): [2.   2.48 2.96 3.44 3.92 4.4 ]
Feature petal length (cm): [1.   2.18 3.36 4.54 5.72 6.9 ]
Feature petal width (cm): [0.1  0.58 1.06 1.54 2.02 2.5 ]

Equal Frequency Discretization:
Feature sepal length (cm): [4.3  5.   5.6  6.1  6.52 7.9 ]
Feature sepal width (cm): [2.  2.7 3.  3.1 3.4 4.4]
Feature petal length (cm): [1.   1.5  3.9  4.64 5.32 6.9 ]
Feature petal width (cm): [0.1  0.2  1.16 1.5  1.9  2.5 ]


In [21]:
#13

import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

transactions = [
    ['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
    ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
    ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
    ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
    ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']
]

df = pd.get_dummies(pd.DataFrame(transactions).stack()).groupby(level = 0).sum()
df = df.astype(bool)

frequent_item = apriori(df, min_support = 0.1, use_colnames = True)
rules_value = association_rules(frequent_item, metric='confidence', min_threshold = 0.5)

print("Frequent Itemsets:\n", frequent_item)
print("\nAssociation Rules:\n", rules_value)

Frequent Itemsets:
      support                                           itemsets
0        0.2                                            (Apple)
1        0.4                                             (Corn)
2        0.2                                             (Dill)
3        0.8                                             (Eggs)
4        0.2                                        (Ice cream)
..       ...                                                ...
144      0.4        (Onion, Nutmeg, Kidney Beans, Eggs, Yogurt)
145      0.2                (Onion, Nutmeg, Eggs, Milk, Yogurt)
146      0.2        (Onion, Nutmeg, Kidney Beans, Milk, Yogurt)
147      0.2  (Onion, Dill, Nutmeg, Kidney Beans, Eggs, Yogurt)
148      0.2  (Onion, Nutmeg, Kidney Beans, Eggs, Milk, Yogurt)

[149 rows x 2 columns]

Association Rules:
            antecedents                                consequents  \
0              (Apple)                                     (Eggs)   
1              (Apple)       

In [ ]:
#Done